In [1]:
import yfinance as yf
import pandas as pd
import pickle
from statsmodels.tsa.arima.model import ARIMA


In [16]:
symbol = "GLD"

In [17]:
start_date = "2004-01-01"
end_date = "2025-2-05"

In [18]:
data = yf.download(symbol, start=start_date, end=end_date)

[*********************100%***********************]  1 of 1 completed


In [19]:
data

Price,Close,High,Low,Open,Volume
Ticker,GLD,GLD,GLD,GLD,GLD
Date,,,,,
2004-11-18,44.380001,44.490002,44.070000,44.430000,5992000
2004-11-19,44.779999,44.919998,44.470001,44.490002,11655300
2004-11-22,44.950001,44.970001,44.740002,44.750000,11996000
2004-11-23,44.750000,44.919998,44.720001,44.880001,3169200
2004-11-24,45.049999,45.049999,44.790001,44.930000,6105100
...,...,...,...,...,...
2025-01-29,254.199997,254.820007,253.300003,254.550003,5671800
2025-01-30,258.049988,258.290009,256.450012,257.200012,12981200


In [20]:
def fetch_gold_price():
    gold_data = yf.download("GC=F", start="2004-01-01", end="2025-02-05")  # XAU/USD 선물 데이터
    gold_data = gold_data[['Close']]  # 조정 종가만 사용
    gold_data.rename(columns={'Close': 'Price'}, inplace=True)
    return gold_data

In [21]:
df = fetch_gold_price()

[*********************100%***********************]  1 of 1 completed


In [22]:
df

Price,Price
Ticker,GC=F
Date,
2004-01-05,424.399994
2004-01-06,422.799988
2004-01-07,421.899994
2004-01-08,424.000000
2004-01-09,426.399994
...,...
2025-01-29,2769.100098
2025-01-30,2823.000000


In [23]:
df = fetch_gold_price()

[*********************100%***********************]  1 of 1 completed


In [24]:
print(df.head())

Price            Price
Ticker            GC=F
Date                  
2004-01-05  424.399994
2004-01-06  422.799988
2004-01-07  421.899994
2004-01-08  424.000000
2004-01-09  426.399994


In [25]:
df.isna().sum()

,,0
Price,Ticker,
Price,GC=F,0


In [26]:
p_values = range(0, 5+1)
d_values = range(0, 2+1)
q_values = range(0, 5+1)

In [27]:
best_aic = float("inf")

In [28]:
best_order = None

In [30]:
import itertools

for p, d, q in itertools.product(p_values, d_values, q_values):
    try:
        model = ARIMA(df['Price'], order=(p, d, q))
        model_fit = model.fit()
        aic = model_fit.aic  # AIC 값 확인

        if aic < best_aic:
            best_aic = aic
            best_order = (p, d, q)

    except:
        continue

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [31]:
print(f"최적의 (p, d, q): {best_order}, 최적 AIC: {best_aic}")

최적의 (p, d, q): (5, 2, 5), 최적 AIC: 43541.012548394865


In [32]:
p, d, q = 5, 2, 5
model = ARIMA(df['Price'], order=(p, d, q))
model_fit = model.fit()

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [34]:
import joblib

joblib.dump(model_fit, "gold_price_arima.pkl")

['gold_price_arima.pkl']